In [1]:
upstream = ['01-get']
product = None

import re

In [2]:
# Parameters
upstream = {"01-get": {"nb": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/kaggle/products/extract-pipeline.ipynb", "csv_file": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/kaggle/data/house_prices.csv"}}
product = {"nb": "/home/luiscberrocal/PycharmProjects/hacktoberfest-2023/kaggle/products/cleanup.ipynb"}


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv(upstream['01-get']['csv_file'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             29451 non-null  int64  
 1   POSTED_BY              29451 non-null  object 
 2   UNDER_CONSTRUCTION     29451 non-null  int64  
 3   RERA                   29451 non-null  int64  
 4   BHK_NO.                29451 non-null  int64  
 5   BHK_OR_RK              29451 non-null  object 
 6   SQUARE_FT              29451 non-null  float64
 7   READY_TO_MOVE          29451 non-null  int64  
 8   RESALE                 29451 non-null  int64  
 9   ADDRESS                29451 non-null  object 
 10  LONGITUDE              29451 non-null  float64
 11  LATITUDE               29451 non-null  float64
 12  TARGET(PRICE_IN_LACS)  29451 non-null  float64
dtypes: float64(4), int64(6), object(3)
memory usage: 2.9+ MB


In [6]:
df.isna().sum()

## %%
renamed_mapping = {}
for c in df.columns:
    new_c_name = re.sub('[^0-9a-zA-Z_]+', '', c)
    renamed_mapping[c] = new_c_name.lower()  # .replace(' ', '_').replace('(', '').replace(')', '')
renamed_mapping['targetprice_in_lacs'] = 'target_price_in_lacs'

df = df.rename(columns=renamed_mapping)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   unnamed0             29451 non-null  int64  
 1   posted_by            29451 non-null  object 
 2   under_construction   29451 non-null  int64  
 3   rera                 29451 non-null  int64  
 4   bhk_no               29451 non-null  int64  
 5   bhk_or_rk            29451 non-null  object 
 6   square_ft            29451 non-null  float64
 7   ready_to_move        29451 non-null  int64  
 8   resale               29451 non-null  int64  
 9   address              29451 non-null  object 
 10  longitude            29451 non-null  float64
 11  latitude             29451 non-null  float64
 12  targetprice_in_lacs  29451 non-null  float64
dtypes: float64(4), int64(6), object(3)
memory usage: 2.9+ MB
